In [1]:
!nvidia-smi

Thu May 26 08:07:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!git clone https://github.com/facebookresearch/fairseq.git
!cd fairseq && pip install .
!pip install subword_nmt sacremoses
!git clone https://github.com/Szuumii/en-de-backtranslation.git
!pip install -q wandb

fatal: destination path 'fairseq' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/fairseq
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Using cached hydra_core-1.0.7-py3-none-any.whl (123 kB)
  Using cached sacrebleu-2.1.0-py3-none-any.whl (92 kB)
  Using cached omegaconf-2.0.6-py3-none-any.whl (36 kB)
  Using cached bitarray-2.5.1-cp37-cp37m-manylinux_2_17_x8

In [3]:
#3863de247ad4f4556935cb0c04f181dba98fc7cf
!wandb login

wandb: Currently logged in as: szumi. Use `wandb login --relogin` to force relogin


In [4]:
# generate dataset for training model for backtranslation
!fairseq-preprocess --source-lang de --target-lang en \
  --trainpref ./en-de-backtranslation/data/datasets/unaugmented/original.train \
  --validpref ./en-de-backtranslation/data/datasets/unaugmented/original.val  \
  --testpref  ./en-de-backtranslation/data/datasets/unaugmented/original.test  \
  --destdir ./data/binarized/ua.tokenized.de-en \
  --thresholdsrc 2 \
  --thresholdtgt 2

2022-05-26 08:09:37 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-05-26 08:09:37 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='./data/binarized/ua.tokenized.de-en', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path='

In [5]:
# train de-en model for backtranslation
!CHECKPOINT_DIR=checkpoints_de_en && fairseq-train ./data/binarized/ua.tokenized.de-en --fp16\
  --arch transformer \
  --source-lang de --target-lang en \
  --dropout 0.1 \
  --attention-dropout 0.1 \
  --activation-dropout 0.1 \
  --encoder-embed-dim 256 \
  --encoder-ffn-embed-dim 512 \
  --encoder-layers 3 \
  --encoder-attention-heads 8 \
  --encoder-learned-pos \
  --decoder-embed-dim 256 \
  --decoder-ffn-embed-dim 512 \
  --decoder-layers 3 \
  --decoder-attention-heads 8 \
  --no-epoch-checkpoints \
  --decoder-learned-pos \
  --max-epoch 10\
  --optimizer adam \
  --lr 5e-4 \
  --batch-size 128 \
  --seed 1 \
  --wandb-project "en-de-backtranslation" \
  --save-dir $CHECKPOINT_DIR \
  #--finetune-from-model ./checkpoints_de_en/checkpoint_last.pt

2022-05-26 08:11:24 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-05-26 08:11:26 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': 'en-de-backtranslation', 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'r

In [6]:
# generate backtranslations of training model with de to en model
! fairseq-generate ./data/binarized/ua.tokenized.de-en \
    --path checkpoints_de_en/checkpoint_best.pt \
    --batch-size 256 \
    --beam 5 \
    --seed 1 \
    --wandb-project "en-de-backtranslation" \
    --results-path saveddeen \
    --gen-subset train  \

2022-05-26 08:41:14 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-05-26 08:41:15 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': 'en-de-backtranslation', 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False

In [7]:
# generate dataset for training baseline model 
!fairseq-preprocess --source-lang en --target-lang de \
  --trainpref ./en-de-backtranslation/data/datasets/unaugmented/original.train \
  --validpref ./en-de-backtranslation/data/datasets/unaugmented/original.val  \
  --testpref  ./en-de-backtranslation/data/datasets/unaugmented/original.test  \
  --destdir ./data/binarized/ua.tokenized.en-de \
  --thresholdsrc 2 \
  --thresholdtgt 2

2022-05-26 09:16:32 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-05-26 09:16:33 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='./data/binarized/ua.tokenized.en-de', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path='

In [18]:
# train en-de baseline model
!CHECKPOINT_DIR=checkpoints_en_de && fairseq-train ./data/binarized/ua.tokenized.en-de --fp16\
  --arch transformer \
  --source-lang en --target-lang de \
  --dropout 0.1 \
  --attention-dropout 0.1 \
  --activation-dropout 0.1 \
  --encoder-embed-dim 256 \
  --encoder-ffn-embed-dim 512 \
  --encoder-layers 3 \
  --encoder-attention-heads 8 \
  --encoder-learned-pos \
  --decoder-embed-dim 256 \
  --decoder-ffn-embed-dim 512 \
  --decoder-layers 3 \
  --decoder-attention-heads 8 \
  --no-epoch-checkpoints \
  --decoder-learned-pos \
  --max-epoch 15\
  --optimizer adam \
  --lr 5e-4 \
  --batch-size 128 \
  --seed 1 \
  --wandb-project "en-de-backtranslation" \
  --save-dir $CHECKPOINT_DIR \
  --finetune-from-model ./checkpoints_de_en/checkpoint_last.pt

2022-05-26 11:08:04 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-05-26 11:08:06 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': 'en-de-backtranslation', 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'r

In [19]:
# evaluate baseline model
! fairseq-generate ./data/binarized/ua.tokenized.en-de \
    --path checkpoints_en_de/checkpoint_best.pt \
    --batch-size 256 \
    --beam 5 \
    --seed 1 \
    --wandb-project "en-de-backtranslation" \
    --results-path saved \
    --scoring bleu

!grep ^H ./saved/generate-test.txt | cut -f3- > gen.out.sys
!grep ^T ./saved/generate-test.txt | cut -f2- > gen.out.ref

!fairseq-score --sys gen.out.sys --ref gen.out.ref

2022-05-26 11:18:54 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-05-26 11:18:56 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': 'en-de-backtranslation', 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False

#### wytrenuj model en-de jeszcze raz po manualnym dodaniu backtranslacji wygenerowanychh przez model de-en do zbioru unaugmented

In [20]:
!grep ^S ./saveddeen/generate-train.txt | cut -f2- > train_src.txt 
!grep ^H ./saveddeen/generate-train.txt | cut -f3- > train_hyps.txt
!grep ^T ./saveddeen/generate-train.txt | cut -f2- > train_trgs.txt

In [21]:
!cat train_hyps.txt >> ./en-de-backtranslation/data/datasets/unaugmented/original.train.en
!cat train_src.txt >> ./en-de-backtranslation/data/datasets/unaugmented/original.train.de

In [22]:
!cat ./en-de-backtranslation/data/datasets/unaugmented/original.train.de | wc -l

399999


In [23]:
!cat ./en-de-backtranslation/data/datasets/unaugmented/original.train.de | wc -l

399999


In [24]:
!fairseq-preprocess --source-lang en --target-lang de \
  --trainpref ./en-de-backtranslation/data/datasets/unaugmented/original.train \
  --validpref ./en-de-backtranslation/data/datasets/unaugmented/original.val  \
  --testpref  ./en-de-backtranslation/data/datasets/unaugmented/original.test  \
  --destdir ./data/binarized/augmented.tokenized.en-de \
  --thresholdsrc 2 \
  --thresholdtgt 2

2022-05-26 11:35:31 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-05-26 11:35:31 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='./data/binarized/augmented.tokenized.en-de', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma

In [25]:
# train en-de model with backtranslation
!CHECKPOINT_DIR=checkpoints_en_de_with_backtranslations && fairseq-train ./data/binarized/augmented.tokenized.en-de --fp16\
  --arch transformer \
  --source-lang en --target-lang de \
  --dropout 0.1 \
  --attention-dropout 0.1 \
  --activation-dropout 0.1 \
  --encoder-embed-dim 256 \
  --encoder-ffn-embed-dim 512 \
  --encoder-layers 3 \
  --encoder-attention-heads 8 \
  --encoder-learned-pos \
  --decoder-embed-dim 256 \
  --decoder-ffn-embed-dim 512 \
  --decoder-layers 3 \
  --decoder-attention-heads 8 \
  --no-epoch-checkpoints \
  --decoder-learned-pos \
  --max-epoch 15\
  --optimizer adam \
  --lr 5e-4 \
  --batch-size 128 \
  --seed 1 \
  --wandb-project "en-de-backtranslation" \
  --save-dir $CHECKPOINT_DIR \
  #--finetune-from-model ./checkpoints_de_en_backtranslation /checkpoint_last.pt

Streaming output truncated to the last 5000 lines.
|       from large pool |       9    |       9    |     190    |     181    |
|       from small pool |       8    |      48    |   22693    |   22685    |
|---------------------------------------------------------------------------|
| Non-releasable allocs |      28    |      44    |   15506 K  |   15506 K  |
|       from large pool |      18    |      19    |    7609 K  |    7609 K  |
|       from small pool |      10    |      36    |    7896 K  |    7896 K  |
|---------------------------------------------------------------------------|
| Oversize allocations  |       0    |       0    |       0    |       0    |
|---------------------------------------------------------------------------|
| Oversize GPU segments |       0    |       0    |       0    |       0    |
|===========================================================================|

2022-05-26 12:59:30 | WARNING | fairseq.trainer | attempting to recover from OOM in forwar

In [26]:
# evaluate model trained with backtranslations
! fairseq-generate ./data/binarized/augmented.tokenized.en-de \
    --path checkpoints_en_de_with_backtranslations/checkpoint_best.pt \
    --batch-size 256 \
    --beam 5 \
    --seed 1 \
    --wandb-project "en-de-backtranslation" \
    --results-path saved_back \
    --scoring bleu

2022-05-26 13:17:53 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-05-26 13:17:54 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': 'en-de-backtranslation', 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False

In [28]:
!grep ^H ./saved_back/generate-test.txt | cut -f3- > gen.out.sys
!grep ^T ./saved_back/generate-test.txt | cut -f2- > gen.out.ref

!fairseq-score --sys gen.out.sys --ref gen.out.ref

2022-05-26 13:36:46 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
Namespace(ignore_case=False, order=4, ref='gen.out.ref', sacrebleu=False, sentence_bleu=False, sys='gen.out.sys')
BLEU4 = 9.44, 38.4/15.6/7.4/3.7 (BP=0.834, ratio=0.847, syslen=815198, reflen=963015)
